In [1]:
print('Hello LangChain')

Hello LangChain


In [7]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI 

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
#print(OPENAI_API_KEY)

In [8]:
# prompt
prompt = ChatPromptTemplate.from_messages(
    [ ("system", "당신은 개발자입니다.") , 
     ("human", "{input}") ]
)
print(prompt)

prompt_text = prompt.format(input="LangServe는 무엇인가요? 자세하게 설명해주세요")

input_variables=['input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 개발자입니다.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]


In [ ]:
# Groq API를 사용하는 ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    #model="mistral-saba-24b",
    temperature=0.7
)
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x000001FEB103B860> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001FEB103A330> root_client=<openai.OpenAI object at 0x000001FEB103AEA0> root_async_client=<openai.AsyncOpenAI object at 0x000001FEB103A720> model_name='mistral-saba-24b' temperature=0.7 model_kwargs={} openai_api_key=SecretStr('**********') openai_api_base='https://api.groq.com/openai/v1'


In [11]:
try:
    response = llm.invoke(prompt_text)
    print(type(response))
    print(response)
    print("응답:", response.content)
except Exception as e:
    print(f"오류 발생: {e}")

오류 발생: Error code: 400 - {'error': {'message': 'The model `mistral-saba-24b` requires terms acceptance. Please have the org admin accept the terms at https://console.groq.com/playground?model=mistral-saba-24b', 'type': 'invalid_request_error', 'code': 'model_terms_required'}}


### LCEL
* Prompt + LLM을 Chain으로 연결하기

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """
    You are an expert in AI Expert. Answer the question. 
    <Question>: {input}에 대해 쉽게 설명해주세요.")
    """)                                     
prompt

PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='You are an expert in AI Expert. Answer the question. <Question>: {input}에 대해 쉽게 설명해주세요.')